<a href="https://colab.research.google.com/github/fatemafaria142/Automated-Realistic-Story-Generation/blob/main/Story_Generation_using_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Dataset Link: https://huggingface.co/datasets/AtlasUnified/atlas-storyteller?row=42**


In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset

dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Display dataset information
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})


In [5]:
# Access the 'train' split of the dataset
train_data = dataset['train']

# Display information about the 'train' split
print(train_data)

Dataset({
    features: ['id', 'Story'],
    num_rows: 5018
})


In [6]:
# Display the first few rows of the 'train' split
print(train_data[:5])  # Displaying the first 5 rows as an example

{'id': ['seed_task_0', 'seed_task_1', 'seed_task_2', 'seed_task_3', 'seed_task_4'], 'Story': ['In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan\'s blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment\'s hesitation, Ethan gathered his meager belongings and 

In [7]:
# Convert the 'train' split to a 'data' CSV file
csv_filename = "/content/drive/MyDrive/Story Generation/data.csv"
train_data.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' has been created.")

Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CSV file '/content/drive/MyDrive/Story Generation/data.csv' has been created.


In [8]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Story Generation/data.csv")
dataset.head()

,id,Story
0,seed_task_0,"In the bustling city of New York, where the ne..."
1,seed_task_1,"The sky stretched out before them, an endless ..."
2,seed_task_2,In the midst of a bustling metropolis teeming ...
3,seed_task_3,"In the heart of a once-thriving metropolis, no..."
4,seed_task_4,In the gritty underbelly of a city plagued by ...


# **Length of the main dataset**

In [9]:
len(dataset)

5018

# **Taking 90% for train dataset and 10% for test dataset**

In [10]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset from the CSV file
data = pd.read_csv("/content/drive/MyDrive/Story Generation/data.csv")

def build_text_files(data_csv, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for index, row in data_csv.iterrows():
        summary = str(row['Story']).strip()
        summary = re.sub(r"[\[\]\"]", "", summary)  # Remove square brackets and quotation marks
        summary = re.sub(r"\s+", " ", summary)  # Replace multiple spaces with a single space
        data += summary + "  "
    f.write(data)

train, test = train_test_split(data, test_size=0.10)

build_text_files(train, '/content/drive/MyDrive/Story Generation/train_dataset.txt')
build_text_files(test, '/content/drive/MyDrive/Story Generation/test_dataset.txt')

print("Train dataset length:", len(train))
print("Test dataset length:", len(test))


Train dataset length: 4516
Test dataset length: 502


# **Text Preprocessing**

In [11]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        for i, example in enumerate(examples[:num_examples]):
            print(f"Example {i+1}: {example}")

train_file_path = '/content/drive/MyDrive/Story Generation/train_dataset.txt'
test_file_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

print("Examples from the Train Dataset:")
read_text_file(train_file_path)

print("\nExamples from the Test Dataset:")
read_text_file(test_file_path)


Examples from the Train Dataset:
Example 1: In the dimly lit streets of a bustling city, Detective Alex Walker slipped quietly through the shadows, his heart pounding with anticipation. His mission: to infiltrate the sinister web of a corrupt religious organization known as The Sanctum. The organization, which masqueraded as a beacon of spirituality, was rumored to be involved in illicit activities behind closed doors. Determined to expose their true nature, Alex had adopted the identity of a devoted disciple seeking enlightenment. Disguised in humble attire, Alex roamed the streets, careful to avoid any suspicious glances that might betray his true purpose. As he made his way to the entrance of The Sanctum, he couldn't help but marvel at the grandeur of the building before him. Its gray stone walls stood tall, adorned with intricate carvings depicting scenes of divine devotion. Entering the premises, Alex found himself in an ethereal sanctuary, illuminated by a cascade of sunlight str

# **GPT2 and its tokenizer**
### **https://huggingface.co/gpt2**

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

In [13]:
train_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'
test_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

# **Prepare the dataset and build a TextDataset**
* The next step is to extract the instructions from all recipes and build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library.

* The block_size parameter is used to specify the maximum length of each sequence or block of text. Text longer than this length will be divided into consecutive blocks of this specified size.

* For example, if we set block_size=128, the text will be processed in blocks of 128 tokens. Any sequence longer than 128 tokens will be divided into smaller consecutive blocks of 128 tokens each.

In [14]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [15]:
!pip install transformers[torch]

In [16]:
!pip install accelerate -U

In [17]:
!pip install --upgrade transformers

# **1st time train - Initialize Trainer with TrainingArguments**

In [18]:
'''
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-storygeneration", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 500, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
'''

In [19]:
'''
trainer.train()
'''

Step,Training Loss
500,2.946000
1000,2.409000


TrainOutput(global_step=1020, training_loss=2.670274087494495, metrics={'train_runtime': 1102.0508, 'train_samples_per_second': 29.373, 'train_steps_per_second': 0.926, 'total_flos': 2114505768960000.0, 'train_loss': 2.670274087494495, 'epoch': 10.0})

In [20]:
'''
trainer.save_model()
'''

# **Loading the saved model and train it again**

* The Trainer class provides an API for feature-complete training. It is used in most of the example scripts from Huggingface. Before we can instantiate our Trainer we need to download our GPT-2 model and create a TrainingArguments to access all the points of customization during training. In the TrainingArguments, we can define the Hyperparameters we are going to use in the training process like our learning_rate, num_train_epochs, or per_device_train_batch_size.


In [21]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = "./gpt2-storygeneration"  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-storygeneration",  # The output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=10,  # Number of training epochs
    per_device_train_batch_size=32,  # Batch size for training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    eval_steps=500,  # Number of update steps between two evaluations
    save_steps=800,  # Save model checkpoint after # steps
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Initialize Trainer with your loaded model and training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


# **Train the model**

In [22]:
# Start training to continue from where it left off
trainer.train()

Step,Training Loss
500,2.225700
1000,1.985000


Checkpoint destination directory ./gpt2-storygeneration/checkpoint-800 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1020, training_loss=2.1023019229664524, metrics={'train_runtime': 1099.8239, 'train_samples_per_second': 29.432, 'train_steps_per_second': 0.927, 'total_flos': 2114505768960000.0, 'train_loss': 2.1023019229664524, 'epoch': 10.0})

# **Save the model**

In [23]:
# Save the model after training
trainer.save_model("./gpt2-englishchef")

# **Test the model**
* To test the model we are going to use another highlight of the transformers library called pipeline. Pipelines are objects that offer a simple API dedicated to several tasks, among others also text-generation

In [24]:
from transformers import pipeline

In [28]:
storyteller = pipeline('text-generation',model='./gpt2-storygeneration', tokenizer='gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
storyteller('In the bustling city of New York,')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In the bustling city of New York, Benjamin, with his unwavering dedication, had found himself embroiled in a delicate and often tragic tale. He had been awarded a prestigious honor, the privilege of being invited to the prestigious ceremony at the prestigious Benjamin Franklin'}]

In [30]:
storyteller(' In the midst of a bustling metropolis teeming with skyscrapers that seemingly ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' In the midst of a bustling metropolis teeming with skyscrapers that seemingly iced off his nose, the young programmer sat at a keyboard, his mind clouded by a world where time became everything. As the first pages of his code flowed'}]

In [32]:
storyteller('In the heart of a once-thriving metropolis, now reduced to a desolate wasteland')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In the heart of a once-thriving metropolis, now reduced to a desolate wasteland, lived a young woman named Maya, whose heart had been ravaged by genetic mutations. Intrigued by her journey, Maya vowed to unlock the secrets she'}]

In [33]:
storyteller('In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. With his sleek black suits and fiery red eyes, he wore his detective badge on his every move. It was said that'

# **Evaluate perplexity on the test dataset**

In [27]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 1.698707938194275


# **Generating Stories without any prompt**

In [54]:
from transformers import pipeline

# Initialize the text generation pipeline
storyteller = pipeline('text-generation', model='./gpt2-storygeneration', tokenizer='gpt2')

# Function to generate stories and save them
def generate_stories(stories_pipeline, num_stories: int):
    generated_stories = []
    for _ in range(num_stories):
        generated_story = stories_pipeline("Story: ")  # You can modify the initial prompt if needed
        generated_stories.append(generated_story[0]['generated_text'])
    return generated_stories

# Generate stories
num_stories_to_generate = 5  # Define the number of stories to generate
generated_stories = generate_stories(storyteller, num_stories_to_generate)

# Display the generated stories
for i, stories in enumerate(generated_stories, start=1):
    print(f"Generated Story {i}:\n{stories}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Story 1:
Story:  Amelia's name echoed through the corridors of Eldoria, as we delved into the secrets that awaited her. Amelia lived in a small rural village named Eldoria. Its streets were lined with winding paths and quaint quaint homes. It was

Generated Story 2:
Story:  It was a dimly lit, dilapidated building, a maze-like maze. Its doors were heavy, and to enter its crumbling halls, you had to find a way home. It was a place where the stories were whispered

Generated Story 3:
Story:  In the quiet town of Oakwood, nestled on a serene bay, a sense of unease hung heavy in the air. The constant tension and uncertainty had gripped the lives of both the citizens and the wealthy, leaving them doub

Generated Story 4:
Story:  In the bustling streets of New York City, where towering skyscrapers pierced the sky and the constant whirring of machinery breathed life into the once-happy city, lived a man named James. With a jagged black beard and

Generated Story 5:
Story:  In the he

# **List of 1st 20 examples from Test Dataset**

In [39]:
def read_text_file(file_path, num_examples=5):
    with open(file_path, 'r') as file:
        content = file.read()
        examples = content.split('  ')  # Assuming '  ' is the separator between examples
        return examples[:num_examples]

test_file_path = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

print("\nExamples from the Test Dataset:")
first_20_examples = read_text_file(test_file_path, num_examples=20)

# Display the first 20 examples
for i, example in enumerate(first_20_examples, start=1):
    print(f"Example {i}: {example}")

# Store the first 20 examples in a list
list_of_20_examples = first_20_examples


Examples from the Test Dataset:
Example 1: In the humble town of Newbridge, nestled in the heartland of the countryside, there lived a peculiar man named Barnaby Grimsby. Tall and hunched, with a shock of unruly gray hair and a perpetual crinkle of mischief in his eyes, Barnaby was the local satirist. With his sharp wit and cutting tongue, he sought to expose the follies and absurdities of society. But it was not just any society that captured Barnaby's satirical commentary—it was the glaring social inequality and rampant poverty that plagued the town. As the sun rose on another day in Newbridge, Barnaby set out on his mission of enlightenment. He wore a tattered coat, threadbare pants, and mismatched shoes, a deliberate choice to mirror the destitution that plagued so many. Armed with his trusty quill and boundless imagination, he roamed the cobblestone streets, observing the townsfolk and seeking inspiration for his biting commentary. At the center of Newbridge stood a grand mansion

In [40]:
print(list_of_20_examples)

["In the humble town of Newbridge, nestled in the heartland of the countryside, there lived a peculiar man named Barnaby Grimsby. Tall and hunched, with a shock of unruly gray hair and a perpetual crinkle of mischief in his eyes, Barnaby was the local satirist. With his sharp wit and cutting tongue, he sought to expose the follies and absurdities of society. But it was not just any society that captured Barnaby's satirical commentary—it was the glaring social inequality and rampant poverty that plagued the town. As the sun rose on another day in Newbridge, Barnaby set out on his mission of enlightenment. He wore a tattered coat, threadbare pants, and mismatched shoes, a deliberate choice to mirror the destitution that plagued so many. Armed with his trusty quill and boundless imagination, he roamed the cobblestone streets, observing the townsfolk and seeking inspiration for his biting commentary. At the center of Newbridge stood a grand mansion, belonging to Lord Percival Kingston, a s

In [41]:
print(len(list_of_20_examples))

20


# **Generating Stories with prompts**

In [57]:
from transformers import pipeline

test_dataset_file = '/content/drive/MyDrive/Story Generation/test_dataset.txt'

# Initialize the text generation pipeline
storyteller = pipeline('text-generation', model='./gpt2-storygeneration', tokenizer='gpt2')

# Read your test dataset
with open(test_dataset_file, 'r') as file:
    test_content = file.read()

# Split the content into individual stories based on the delimiter ('  ')
test_stories = test_content.split('  ')

# Function to generate stories based on the test dataset
def generate_stories_from_test_dataset(story_pipeline, test_dataset, num_stories: int):
    generated_stories = []
    for i in range(num_stories):
        # Extract the first 6 words from the stories as the prompt
        words = test_dataset[i].split()[:6] if i < len(test_dataset) else test_dataset[-1].split()[:6]
        prompt = ' '.join(words)
        generated_story = story_pipeline(prompt, max_length=128)  # Adjust max_length as needed
        generated_stories.append(generated_story[0]['generated_text'])
    return generated_stories

# Generate stories based on the test dataset
num_stories_to_generate = 20  # Define the number of stories to generate
generated_stories_from_test = generate_stories_from_test_dataset(storyteller, test_stories, num_stories_to_generate)

# Display the generated stories
for i, stories in enumerate(generated_stories_from_test, start=1):
    print(f"Generated Story {i}:\n{stories}\n")

# Save the generated stories in a list
list_of_generated_stories = generated_stories_from_test


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Generated Story 1:
In the humble town of Newbridge, nestled amidst rolling hills and picturesque forests, nestled among rolling hills and picturesque woods, lived a young woman named Lily. With a radiant smile and a fiery determination, she possessed an insatiable desire to excel academically and excel in her art. Lily, renowned for her poetic wit and unwavering determination, always found solace in her reading friends' hearts, seeking the courage to embark on this arduous academic journey. The town of Newbridge, nestled amidst rolling hills and picturesque forests, lay nestled amongst rolling hills and picturesque woods, where life had always seemed

Generated Story 2:
In a small town nestled amidst rolling hills and meandering streams, there lived a young woman named Amelia. With her flowing flowing green hair and piercing blue eyes, Amelia had always felt a flicker of excitement in her heart as she ventured into the realm of adventure. Amelia was known for her insatiable curiosity a

In [60]:
print(list_of_generated_stories)

["In the humble town of Newbridge, nestled amidst rolling hills and picturesque forests, nestled among rolling hills and picturesque woods, lived a young woman named Lily. With a radiant smile and a fiery determination, she possessed an insatiable desire to excel academically and excel in her art. Lily, renowned for her poetic wit and unwavering determination, always found solace in her reading friends' hearts, seeking the courage to embark on this arduous academic journey. The town of Newbridge, nestled amidst rolling hills and picturesque forests, lay nestled amongst rolling hills and picturesque woods, where life had always seemed", 'In a small town nestled amidst rolling hills and meandering streams, there lived a young woman named Amelia. With her flowing flowing green hair and piercing blue eyes, Amelia had always felt a flicker of excitement in her heart as she ventured into the realm of adventure. Amelia was known for her insatiable curiosity and tireless pursuit of knowledge. 

In [61]:
print(len(list_of_generated_stories))

20


# **Evaluating Generated Recipes Against Reference Recipes**

In [46]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=830cb89c8561c89ed0077c5687714f0df04108f1bdded756a004c966d7d9806a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [47]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 944.2 kB/s eta 0:00:00


In [48]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 11.0 MB/s eta 0:00:00


In [49]:
from evaluate import load
# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [62]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)
results_WER = wer_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)
results_met = meteor.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# Calculate Bilingual Evaluation Understudy (BLEU)
results_bleu = bleu_metric.compute(predictions=list_of_generated_stories, references=list_of_20_examples)

# **Printing evaluation metrics**

In [63]:
print("Character Error Rate for Story Generation:", results_CER)
print("Word Error Rate for Story Generation:",results_WER)
print("Exact Match for Story Generation:",results_em)
print("BLEU Score for Story Generation:",results_bleu)
print("METEOR for Story Generation:",results_met)

Character Error Rate for Story Generation: 0.868756579716926
Word Error Rate for Story Generation: 0.9477481416703104
Exact Match for Story Generation: {'exact_match': 0.0}
BLEU Score for Story Generation: {'bleu': 0.0004679039332456458, 'precisions': [0.5592497868712702, 0.17669819432502148, 0.06504770164787511, 0.04286964129483815], 'brevity_penalty': 0.0036316313778264535, 'length_ratio': 0.15110137833311865, 'translation_length': 2346, 'reference_length': 15526}
METEOR for Story Generation: {'meteor': 0.05732767404320359}
